In [1]:
import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp import GasOptics
from pyrte_rrtmgp.rrtmgp_data_files import GasOpticsFiles
import angles


def make_atmosphere(p, T, h2o_vmr, o3, T_s=None):
    """Create a pyRTE-RRTMG atmosphere from pressure, temperature and humidity arrays."""
    if np.any(p[..., 0] < p[..., -1]):
        raise ValueError("Arrays need to be passed in ascending order")

    if T_s is None:
        T_s = T[..., 0]

    atmosphere = xr.Dataset(
        data_vars={
            "pres_level": (("column", "level"), p),
            "temp_level": (("column", "level"), T),
            "pres_layer": (("column", "layer"), 0.5 * (p[..., 1:] + p[..., :-1])),
            "temp_layer": (("column", "layer"), 0.5 * (T[..., 1:] + T[..., :-1])),
            "surface_temperature": (("column",), T_s),
            "h2o": (("column", "layer"), 0.5 * (h2o_vmr[..., 1:] + h2o_vmr[..., :-1])),
            "o3": (("layer"), 0.5 * (o3[..., 1:] + o3[..., :-1])),
            "co2": 422e-6,
            "ch4": 1650e-9,
            "n2o": 306e-9,
            "n2": 0.7808,
            "o2": 0.2095,
            "co": 0.0,
        },
    )

    return atmosphere

In [2]:
ds = xr.open_dataset(
    "/work/mh0066/m301046/ml_clouds/sondes_for_radiation.nc"
).swap_dims({"sonde": "sonde_id"})
lvl3 = xr.open_dataset(
    "ipfs://bafybeiesyutuduzqwvu4ydn7ktihjljicywxeth6wtgd5zi4ynxzqngx4m", engine="zarr"
).swap_dims({"sonde": "sonde_id"})
# ds = ds.pipe(dh.interpolate_gaps).pipe(dh.extrapolate_sfc)
# ds = ds.isel(  # pick every 10th sonde
#    altitude=slice(0, 1100),  # skip upper-most layers due to NaN values
# ).dropna(dim="sonde", how="any", subset=["ta", "q", "p"])

ds = ds.assign(
    launch_lat=lvl3.launch_lat.sel(sonde_id=ds.sonde_id),
    launch_lon=lvl3.launch_lon.sel(sonde_id=ds.sonde_id),
    launch_time=lvl3.launch_time.sel(sonde_id=ds.sonde_id),
)  # .isel(sonde_id=slice(45, 50))

In [3]:
ds = ds.swap_dims({"sonde_id": "sonde"}).assign(
    mu0=xr.apply_ufunc(
        angles.cos_zenith_angle,
        ds.launch_time,
        ds.launch_lat,
        ds.launch_lon,
        vectorize=True,
    )
)

In [4]:
ds.launch_time.values

array(['2024-08-09T14:27:28.000000000', '2024-08-09T15:05:24.000000000',
       '2024-08-11T13:20:01.000000000', ...,
       '2024-09-28T19:04:52.000000000', '2024-09-28T19:09:51.000000000',
       '2024-09-28T19:14:52.000000000'],
      shape=(1069,), dtype='datetime64[ns]')

In [ ]:
atmosphere = make_atmosphere(ds.p.values, ds.t.values, ds.H2O.values, ds.O3.values)
atmosphere

<xarray.Dataset> Size: 8MB
Dimensions:              (column: 1069, level: 180, layer: 179)
Dimensions without coordinates: column, level, layer
Data variables: (12/13)
    pres_level           (column, level) float64 2MB 1.012e+05 ... 103.1
    temp_level           (column, level) float64 2MB 296.4 296.0 ... 269.4 269.9
    pres_layer           (column, layer) float64 2MB 1.009e+05 1e+05 ... 109.5
    temp_layer           (column, layer) float64 2MB 296.2 295.5 ... 268.6 269.6
    surface_temperature  (column) float64 9kB 296.4 298.8 300.6 ... 302.6 302.5
    h2o                  (column, layer) float64 2MB 0.02425 ... 5.92e-06
    ...                   ...
    co2                  float64 8B 0.000422
    ch4                  float64 8B 1.65e-06
    n2o                  float64 8B 3.06e-07
    n2                   float64 8B 0.7808
    o2                   float64 8B 0.2095
    co                   float64 8B 0.0

: 

In [ ]:
gas_optics_lw = GasOptics(gas_optics_file=GasOpticsFiles.LW_G256)
gas_optics_sw = GasOptics(gas_optics_file=GasOpticsFiles.SW_G224)
optical_props = gas_optics_lw.compute(atmosphere, add_to_input=False)

optical_props = optical_props.assign(surface_emissivity=0.98)

In [ ]:
clr_fluxes = optical_props.rte.solve(add_to_input=False)

In [ ]:
optical_props_sw = gas_optics_sw.compute(atmosphere, add_to_input=False)

In [ ]:
optical_props_sw["surface_albedo"] = 0.06
optical_props_sw = optical_props_sw.assign(mu0=("column", ds.mu0.values))
sw_fluxes = optical_props_sw.rte.solve(add_to_input=False)

In [ ]:
atmosphere = atmosphere.assign_coords(
    {
        "sonde_id": ("column", ds.sonde_id.values),
        "launch_time": ("column", ds.launch_time.values),
        "launch_lat": ("column", ds.launch_lat.values),
        "launch_lon": ("column", ds.launch_lon.values),
    }
)

In [ ]:
xr.merge([clr_fluxes, sw_fluxes, atmosphere, optical_props_sw[["mu0"]]]).to_zarr(
    "/scratch/m/m301046/rrtmgp_sonde_fluxes.zarr", mode="w", zarr_format=2
)

In [ ]:
from myutils.physics_helper import make_sounding_from_adiabat
import moist_thermodynamics.functions as mtf
import moist_thermodynamics.saturation_vapor_pressures as svp

Px = ds.p.mean(dim="sonde").sel(altitude=0).values
P = np.arange(Px, 4000.0, -500)
T2 = ds.t.mean(dim="sonde").sel(altitude=0).values
q2 = ds.q.mean(dim="sonde").sel(altitude=0).values  # 9182267570514704

pseudo2 = make_sounding_from_adiabat(P, T2, q2, thx=mtf.theta_e_bolton, Tmin=195)
pseudo2 = pseudo2.assign(
    q=mtf.relative_humidity_to_specific_humidity(
        0.98, pseudo2.P, pseudo2.T, es=svp.liq_wagner_pruss
    )
)

In [ ]:
atmosphere = make_atmosphere(
    pseudo2.P.values.reshape(1, pseudo2.P.shape[0]),
    pseudo2.T.values.reshape(1, pseudo2.P.shape[0]),
    pseudo2.q.values.reshape(1, pseudo2.P.shape[0]),  # wrong!
    o3=ds.O3.interp(altitude=pseudo2.altitude).values,
)
gas_optics_sw = GasOptics(gas_optics_file=GasOpticsFiles.SW_G224)
op_sw = gas_optics_sw.compute(atmosphere, add_to_input=False)
op_sw["surface_albedo"] = 0.06
op_sw = op_sw.assign(mu0=1)
sw_fluxes = op_sw.rte.solve(add_to_input=False)

In [ ]:
xr.merge([sw_fluxes, atmosphere]).to_zarr(
    "/scratch/m/m301046/pseudo_1.zarr", mode="w", zarr_format=2
)

In [ ]:
op_sw = op_sw.assign(mu0=0.5)
sw_fluxes = op_sw.rte.solve(add_to_input=False)
xr.merge([sw_fluxes, atmosphere]).to_zarr(
    "/scratch/m/m301046/pseudo_0.5.zarr", mode="w", zarr_format=2
)